### Map Recipes to Meal Plan
#### Takes : Diet Plan csv , recipe_list csv (dump from datbase), serving unit list csv (dump from db)
1. Take and normalised Recipe CSV
2. Pre process the data by adding unit id and recipe_id columns
3. get list of recipes to be added from the database
4. update meal map recipes in database 

In [ ]:
import pandas as pd
import json
import pickle
import requests
import pathlib
import time
import numpy as np
import os
import requests
from itertools import chain
import ast

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def load_json(path):
    with open(path , 'r') as f:
        return json.load(f)

def export_json(data, path='result.json'):
    with open(path, 'w') as f:
        json.dump(data, f)

def load_pickle(path):
    with open(path , 'rb') as f:
        return pickle.load(f)
    
def export_pickle(data, path='result.pikle'):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

#### Create a mapper for serving unit 
`{ 'unit_name' : 'unit_id' }` 

In [ ]:
def get_serving_unit_mapper(df_units):
    serving_unit_mapper = dict(zip(df_units.serving_unit_name.str.lower(), df_units.id))
    serving_unit_mapper['tbsp'] = serving_unit_mapper['table spoon']
    serving_unit_mapper['tsp'] = serving_unit_mapper['tea spoon']
    serving_unit_mapper['pc'] = serving_unit_mapper['pieces']
    serving_unit_mapper['whole'] = serving_unit_mapper['whole fruit']
    serving_unit_mapper['oz'] = serving_unit_mapper['ounce']
    return serving_unit_mapper

#### Filter and rename columns as per API requirement 

In [ ]:
def plan_api_data_format(df_diet_plan_api):
    cols = ['recipe_id', 'plan_schedule_id', 'unit_id', 'Servings']
    df_diet_plan_api = df_diet_plan_api[cols]

    df_diet_plan_api.rename(columns={'recipe_id': 'recipe_id', 
                                    'plan_schedule_id': 'schedule_id', 
                                    'unit_id': 'serving_unit_id', 
                                    'Servings': 'quantity'}, inplace=True)

    df_diet_plan_api.recipe_id = df_diet_plan_api.recipe_id.astype('int')
    df_diet_plan_api.quantity = df_diet_plan_api.quantity.astype('float')
    df_diet_plan_api.serving_unit_id = df_diet_plan_api.serving_unit_id.astype('int')
    return df_diet_plan_api.to_dict('records')

#### Map recipe standardise name with db name 

In [ ]:
def get_recipe_mapper(df_diet_plan_api, df_units):
    serving_unit_mapper = get_serving_unit_mapper(df_units)
    df_diet_plan_api.rename(columns={'Normalized Name': 'recipe_names'}, inplace=True)
    recipe_id_mapper = dict(zip(df_meal.recipe_name.str.lower(), df_meal.id))
    df_diet_plan_api['recipe_id'] = df_diet_plan_api.recipe_names.str.lower().map(recipe_id_mapper)
    df_diet_plan_api['unit_id'] = df_diet_plan_api.Unit.str.lower().map(serving_unit_mapper)
    df_diet_plan_api.unit_id = df_diet_plan_api.unit_id.fillna(serving_unit_mapper[np.nan])
    return df_diet_plan_api

#### API call to add recipe to meal plan

In [ ]:
def update_meal_plan(payload_results):
    url = "http://18.223.178.105:5000/api/v1/plan_management/planMeal"
    headers = { "Content-Type" : "application/json;charset=utf-8"}
    failed, success = [], []
    for payload in payload_results:
        try:
            payload = json.dumps(payload)
            response = requests.request("POST", url, headers=headers, data=payload)
            print(response.text)
            success.append(payload)
            time.sleep(0.7)
        except Exception as e:
            print('\t Failed ', payload)
            print('\t Error', e)
            failed.append(payload)
    return failed, success

#### get the list of recipes mapped to meal plan 

In [ ]:
def get_plan_meals(plan_id):
    url = f"http://18.223.178.105:5000/api/v1/plan_management/plan/{plan_id}"
    response = requests.request("GET", url, headers={}, data="")
    return json.loads(response.text)

In [ ]:
def get_meal_map_details(plan_id):
    res = get_plan_meals(plan_id)
    temp = []
    for day, vals in res['plan_details'].items():
        for tm, rep in vals.items():
            rep = rep['recipes']
            for obj in rep:
                obj['day_name'] = day
                obj['time'] = tm
                temp.append(obj)
    temp = pd.DataFrame.from_dict(temp)
    temp.recipe_name = temp.recipe_name.str.lower()
    temp = temp.groupby(['day_name', 'time'])['recipe_name'].unique()
    temp = temp.reset_index()
    return temp

In [ ]:
def get_db_meal_plan_details(plan_id):
    res = get_plan_meals(plan_id)
    temp = dict()
    for day, vals in res['plan_details'].items():
        tm_dict = dict()
        for tm, rep in vals.items():
            rep = rep['recipes']
            recipe_id = [x['recipe_id'] for x in rep]
            tm_dict[str.lower(tm)] = recipe_id
        temp[str.lower(day)] = tm_dict
    return temp

In [ ]:
def check_is_recipe_mapped(plan_data, day_label, time_label, recipe_id):
    if plan_data:
        data = plan_data.get(str.lower(day_label), {})
        data = data.get(str.lower(time_label), [])
        return not int(recipe_id) in data
    else:
        return True

# check_is_recipe_mapped(plan_data, 'friday', 'Dinner', 51)

In [ ]:
def add_recipe_to_meal_plan(df_diet_plan):
    failed, success = [], []
    for plan_id, df_diet_plan_api in df_diet_plan.groupby('plan_id'):
        try:
            print('Uploading ... ', plan_id)
            plan_data = get_db_meal_plan_details(plan_id)
            df_diet_plan_api['upload_status'] = df_diet_plan_api.apply(lambda row: check_is_recipe_mapped(plan_data, 
                                                                              row['day_name'], 
                                                                              row['time'], 
                                                                              row['recipe_id']), axis=1)
            df_diet_plan_api = df_diet_plan_api[df_diet_plan_api['upload_status']]

            payload_results = plan_api_data_format(df_diet_plan_api)
            failed_, success_ = update_meal_plan(payload_results)

            success.append(success_)
        except Exception as e:
            print('\t failed to upload ', plan_id)
            failed.append(failed_)
    return failed, success

In [ ]:
def pre_process_recipe_data(df_diet_plan, df_units, df_meal):
    df_meal.recipe_name = df_meal.recipe_name.str.replace(' ', '')
    recipe_id_mapper = dict(zip(df_meal.recipe_name.str.lower(), df_meal.id))

    # recipe id mappping 

    df_diet_plan['Normalized Name'] = df_diet_plan['Name'].str.replace(' ', '')
    df_diet_plan['recipe_id'] = df_diet_plan['Normalized Name'].str.lower().map(recipe_id_mapper)

    # serving unit mapping 
    df_diet_plan = get_recipe_mapper(df_diet_plan, df_units)

    # df_diet_plan = df_diet_plan[df_diet_plan.unit_id !=30]

    missing_df1 = df_diet_plan[(df_diet_plan.recipe_id.isna()) | (df_diet_plan.Servings.isna())]
    missing_df = df_diet_plan[(df_diet_plan.recipe_id.isna()) | (df_diet_plan.Servings.isna())]
    missing_df.to_csv('mis_match_recipes.csv', index=False)
    df_diet_plan = df_diet_plan[df_diet_plan.recipe_id.notna() & (df_diet_plan.Servings.notna())]
    df_diet_plan = df_diet_plan[df_diet_plan.recipe_id.notna() & (df_diet_plan.Servings.notna())]
    return df_diet_plan

1. `df_diet_plan` CSV file path for standardise recipes diet plan
2. `df_meal` : CSV file path for dataase recipes (db dump)
3. `df_serving` : CSV file path for database serving units (db dump)

In [ ]:
df_diet_plan = pd.read_csv('./fooddb/csv/Meal_plan_mapping_format_28_diet_plans_All_plans v4 (Serving unit fixed).csv')
df_meal = pd.read_csv('./fooddb/csv/database_310_meals.csv')
df_units = pd.read_csv('./fooddb/csv/nureca_serving_units.csv')

In [ ]:
df_diet_plan = pre_process_recipe_data(df_diet_plan, df_units, df_meal)
failed, success = add_recipe_to_meal_plan(df_diet_plan)